In [ ]:
!pip install -q datasets torch==1.13.1 transformers==4.30.1 accelerate==0.21.0 bitsandbytes peft trl

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
import torch

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from huggingface_hub import login

login()

In [ ]:
dataset = load_dataset(
  'json',
  data_files="/content/drive/MyDrive/pybr/train.jsonl",
  split="train"
)

In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

In [ ]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = get_peft_model(model,peft_config)

In [ ]:
model.print_trainable_parameters()

In [ ]:
output_dir = "/content/drive/MyDrive/training data/results"

per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:
max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    args=training_arguments,
)

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
model_to_save.save_pretrained("/content/drive/MyDrive/pybr/outputs")

In [ ]:
lora_config = LoraConfig.from_pretrained('/content/drive/MyDrive/pybr/outputs')
model = get_peft_model(model, lora_config)

In [ ]:
text = "Below is a statement that describes a task, along with an input that provides more context. Write a response that adequately completes the request.\n### Instruction:\nMarv is a factual chatbot that is also sarcastic.\n### Input:\nWho is the all-time top scorer for the Brazil national football team?\n### Output:\n "
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)

In [ ]:
model.print_trainable_parameters()

In [ ]:
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))